# Computing the Homography Matrix

This article will show you how to compute the homography matrix between two images of the same object taken from 2 different perspectives.

> See https://araintelligence.com/blogs/computer-vision/geometric-vision/understand-the-homography-matrix for a quick overview of the homography matrix

![Images A (left) and B (right)](ely-cathedral-A-B.png)

I'd show you how to estimate the homography between the 2 cathedral images shown above. We already have a set of 4 point correspondences between the 2 images.
You can either pick these points manually using `ginput` from matplotlib or you can use a keypoint detector like SIFT to detect and match point correspondences between both images (The focus of this article is on the homography matrix so it's ok to pick the points manually).


The homography matrix that defines the mapping between the two images can be defined as follows

$$


\begin{bmatrix}
    x' \\
    y' \\
    1  \\
\end{bmatrix}
=
\alpha
\begin{bmatrix}
h_{1} & h_{2} & h_{3} \\
h_{4} & h_{5} & h_{6} \\
h_{7} & h_{8} & h_{9} 
\end{bmatrix}
\left[ {\begin{array}{c}
    x \\
    y \\
    1  \\
\end{array}} \right] 

\\
\text{ also as } \\
\newline

\bf{x'} = \alpha \bf{H} \bf{x} 

$$

And the problem this article will show you how to solve is that, given a two set of matched points between the two images $\{(x_1,y_1), ..., (x_n,y_n)\}$ and $\{(x'_1,y'_1), ..., (x'_n,y'_n)\}$, how can we estimate the parameters in the $\bf{H}$ matrix

## Estimating the Homography Matrix using the Direct Linear Transform (DLT) Algorithm
Harley and Zisserman describe the DLT algorithm for obtaining an initial estimate of the parameters of the homography matrix as

### Generate the inputs
Construct a pair of the input features as $n$ points $\{(x_1,y_1), ..., (x_n,y_n)\}$ from image A and $n$ points $\{(x'_1,y'_1), ..., (x'_n,y'_n)\}$ from image B.
```python
def construct():
    return 1+1
# code that does not run
```

### Normalise the input data
Before the input features are used to estimate the homography matrix. They are first normalised, this is done to ensure numerical stability. The points are conditioned to have zero mean and unit standard deviation. Each point is standardised using the z-score as
$$
z = \frac{p - \mu}{\sigma}
$$
This can be done in matrix form by constructing a similarity transform, for each set of points, which is a $3 \times 3$ matrix $T$ as
$$
\left[ {\begin{array}{c}
    x \\
    y \\
    1 \\
\end{array}} \right]_A = 
\left[ {\begin{array}{ccc}
\frac{1}{\sigma} & 0 & \frac{-\mu}{\sigma} \\
0 & \frac{1}{\sigma} & \frac{-\mu}{\sigma} \\
0 & 0 & 1 
\end{array}} \right]_A
\left[ {\begin{array}{c}
    x \\
    y \\
    1 \\
\end{array}} \right]_A
$$
for point set A from image A. The same is done for points in image B.

Normalising input data before feeding them through your algorithm is a common pre-processing step in many computer vision and deep learning workflows. It helps to reduce ambiguity in the data by finding a common ground between them i.e zero mean and unit variance. In our image example, images A and B whilst taken from the same camera could otherwise be different, image A could be of size 360x700 and image B could be of size 1200x1200 - In such a case, it helps to normalise the points from both images to improve the accuracy and stability in the next steps of the workflow.

### Construct a system of linear equations
Solve for the parameters of the homography matrix by constructing a system of linear equations as follows. For a given point pair that have been normalised, $\{(x,y),(x',y')\}$ we can start from the homography matrix form defined earlier as
$$
\begin{bmatrix}
x' \\
y' \\
1
\end{bmatrix}

= \alpha
\left[ {\begin{array}{ccc}
h_{1} & h_{2} & h_{3} \\
h_{4} & h_{5} & h_{6} \\
h_{7} & h_{8} & h_{9} 
\end{array}} \right]
\left[ {\begin{array}{c}
    x \\
    y \\
    1  \\
\end{array}} \right]
$$

Which can be written out as

$$
x' = \alpha (h_1 x + h_2 y + h_3) \hspace3em \mathrm{(i)} \\
y' = \alpha (h_4 x + h_5 y + h_6) \hspace3em \mathrm{(ii)} \\
1 = \alpha (h_7 x + h_8 y + h_9) \hspace3em \mathrm{(iii)}
$$

Convert equation $\mathrm{(i)}$ to homogeneous coordinates by dividing equation $\mathrm{(iii)}$ as follows
$$
\frac{x'}{1} = \frac{\alpha (h_1 x + h_2 y + h_3)}{\alpha (h_7 x + h_8 y + h_9)}
$$

> A point $(a,b,w)$ written in homogenous coordinates, is the same as $(\frac{a}{w},\frac{b}{w})$ in cartesian coordinates

The unknown scale factor $\alpha$ cancels out and the terms are rearranged to form
$$
x'(h_7 x + h_8 y + h_9) = h_1 x + h_2 y + h_3 \\
x'x h_7 + x'y h_8 + x'h_9 - xh_1 -yh_2 -h_3 = 0 \\
\text{ re-ordering and adding empty terms } \\
-xh_1 -yh_2 -h_3 + 0h_4 + 0h_5 + 0h_6 + x'xh_7 + x'yh_8 + x'h_9 = 0 \hspace3em \mathrm{(iv)}
$$

Equation $\mathrm{(ii)}$ is also converted to homogeneous coordinates to produce equation $\mathrm{(v)}$ as
$$
y'(h_7x + h_8y + h_9) = h_4x + h_5y + h_6 \\
xy'h_7 + yy'h_8 + y'h_9 - h_4x -h_5y -h_6 = 0 \\
\text{ re-ordering and adding empty terms } \\
0h_1 + 0h_2 + 0h_3 -xh_4 -yh_5 -h_6 + xy'h_7 + y'yh_8 + y'h_9 = 0 \hspace3em \mathrm{(v)}
$$

#### System of equations in the form $Ah = 0$
From here, it's getting obvious that we can derive our system of linear equations from $\mathrm{(iv)}$ and $\mathrm{(v)}$ as follows

$$
\begin{bmatrix}
-xh_1 & -yh_2 & -h_3 & + 0h_4 & + 0h_5 & + 0h_6 & + x'xh_7 & + x'yh_8 & + x'h_9 \\
0h_1 & + 0h_2 & +0h_3 & -xh_4 & -yh_5 & -h_6 & + xy'h_7 & + y'yh_8 & + y'h_9
\end{bmatrix}
=
\begin{bmatrix}
0 \\
0
\end{bmatrix}
$$

This can also be written in matrix form as $A_i\bf{h} = \bf{0}$ where the first point pair ${(x_1,y_1), (x'_1,y'_1)}$
$$
A_i = 
\begin{bmatrix}
-x_1 & -y_1 & -1 & 0 & 0 & 0 & +x'_1x_1 & +x'_1y_1 & +x'_1 \\
0 & 0 & 0 & -x_1 & -y_1 & -1 & +x_1y'_1 & +y'_1y_1 & +y'_1
\end{bmatrix}
\\
\text{ and } \bf{h} = 
\begin{bmatrix}
h_1 \\
h_2 \\
h_3 \\
h_4 \\
h_5 \\
h_6 \\
h_7 \\
h_8 \\
h_9 
\end{bmatrix}
$$

Given that 1 point pair ${(x_1,y_1), (x'_1,y'_1)}$ produces a system of 2 equations in linear form.
To create a determined system of equations, we would need 5 point pairs which will further give us a system of 10 equations (and now an over-determined system of equations).

However, we know that this is a planar 2D transform and our resulting homography matrix **must** have the form
$$
\begin{bmatrix}
h_{1} & h_{2} & h_{3} \\
h_{4} & h_{5} & h_{6} \\
h_{7} & h_{8} & 1 
\end{bmatrix}
$$
Therefore, we set $h_9 = 1$ and now we have only 8 unknown parameters. Therefore, we only need 4 points, which will give a system of 8 linear equations and is a determined system of equations. Therefore, our full system can be constructed as

$$
\begin{bmatrix}
-x_1 & -y_1 & -1 & 0 & 0 & 0 & +x'_1x_1 & +x'_1y_1 & +x'_1 \\
0 & 0 & 0 & -x_1 & -y_1 & -1 & +x_1y'_1 & +y'_1y_1 & +y'_1 \\
\vdots & & & & \ddots & & & & \vdots \\
-x_8 & -y_8 & -1 & 0 & 0 & 0 & +x'_8x_8 & +x'_8y_8 & +x'_8 \\
0 & 0 & 0 & -x_8 & -y_8 & -1 & +x_8y'_8 & +y'_8y_8 & +y'_8 
\end{bmatrix}
=
\begin{bmatrix}
h_1 \\
h_2 \\
h_3 \\
h_4 \\
h_5 \\
h_6 \\
h_7 \\
h_8 \\
h_9 
\end{bmatrix}
=
\begin{bmatrix}
0_1 \\
0_2 \\
0_3 \\
0_4 \\
0_5 \\
0_6 \\
0_7 \\
0_8 
\end{bmatrix}
$$

### Use the Singular Value Decomposition (SVD) of A to find the initial estimate of the Homography H
The matrix A can be decomposed as 
$$
\large A = U \sum V^T
$$
where $\sum$ is a $9\times9$ diagonal matrix of eigenvalues with positive entries that decrease from upper left to lower right. The smallest eigenvalue is the lower right and its corresponding eigenvector would be the last column of $V$. $V$ is a $9 \times 9$ matrix and its last column would be $9\times1$.
The diagonal entries in $\sum$ represent the reprojection error, we choose the last diagonal entry because it would be the smallest reprojection error amongst all entries (eigen values).

> There are other techniques for estimating the homography matrix - The direct linear transform algorithm does not perform well when the corresponding point pairs $\{(x_1,y_1), ..., (x_n,y_n)\}$ from image A and $\{(x'_1,y'_1), ..., (x'_n,y'_n)\}$ from image B contain outliers e.g wrong point matches, deviations in a corresponding point pair etc

The last column vector $\bf{h}$ needs to be reshaped into the initial estimate $\hat{H}$ 
```python
H = V[8].reshape((3,3))
```

The current homography $\hat{H}$ is the homography for the normalised points. To convert it into an homography that understands the coordinate space of both image points, we need to decondition it back using $T'$ and $T$. Therefore
$$
H = T^{-1}_{B} \hat{H} T_{A}
$$
```python
# decondition
H = np.dot(np.linalg.inv(T2), np.dot(H, T1))
```

One last step before returning the homography matrix. Since the projective transform lies on the same $z$ plane, changes in the z-plane should not affect the resulting image as all the image points lie on the same z-plane (See [this video by 3b1b for a visualisation of what is happening](https://www.3blue1brown.com/lessons/3d-transformations)). 

We can simplify the homography matrix by setting $h_{9} = 1$ as this parameter doesn't determine how the points changed in the images but simply performs the visual effect of changing the z-plane which will produce a zoom-in/zoom-out effect.
$$
\begin{bmatrix}
h_{1} & h_{2} & h_{3} \\
h_{4} & h_{5} & h_{6} \\
h_{7} & h_{8} & \bf{1}
\end{bmatrix}
$$
With this fact, we can improve the estimates even further by normalising the entire matrix with $h_9$.
```python
H = H / H[2,2] # normalise matrix H with h9 so that z=1
```
> NB: Only normalise by $h_9$ only when the you're computing the homography between planar transformations.

The entire `homography` function is shown below

In [ ]:
def homography(fp,tp):
    """
    Find homography H, such that points fp are mapped to tp using the linear DLT method
    Points are conditioned automatically (What does this mean?)
    Points are represented as column vectors using the format 3xN
    """

    if fp.shape != tp.shape:
        raise RuntimeError("Number of points do not match")

    # condition points (important for numerical reasons)
    # Que?: How are the points conditioned? Is there any reference that explains why & how they are conditioned
    # from points (fp)
    """
    According to the book, the points are conditioned to have zero mean and unit standard deviation to make 
    the calculations numerically stable.
    """
    m = np.mean(fp[:2], axis=1)
    maxstd = np.max(np.std(fp[:2], axis=1)) + 1e-9
    C1 = np.diag([1/maxstd, 1/maxstd, 1])
    C1[0][2] = -m[0]/maxstd
    C1[1][2] = -m[1]/maxstd
    fp =  np.dot(C1, fp)
    # to points
    m = np.mean(tp[:2], axis=1)
    maxstd = np.max(np.std(tp[:2], axis=1)) + 1e-9
    C2 = np.diag([1/maxstd, 1/maxstd, 1])
    C2[0][2] = -m[0]/maxstd
    C2[1][2] = -m[1]/maxstd
    tp =  np.dot(C2, tp)

    # create matrix for linear method, 2 rows for each correspondence pair
    nbr_correspondences = fp.shape[1]
    A = np.zeros((2*nbr_correspondences))
    for i in range(nbr_correspondences):
        A[2*i] = [-fp[0][i], -fp[1][i], -1,0,0,0, tp[0][i]*fp[0][i], tp[0][i]*fp[1][i], tp[0][i]]
        A[2*i+1] = [0,0,0, -fp[0][i], -fp[1][i], -1,tp[1][i]*fp[0][i], tp[1][i]*fp[1][i], tp[1][i]]
    U,S,V = np.linalg.svd(A)
    H = V[8].reshape((3,3))

    # decondition
    H = np.dot(np.linalg.inv(C2), np.dot(H, C1))

    # normalize and return
    return H / H[2,2] # normalise with H(2,2), maybe because H(2,2) should always be a 1 as z=1

# Computing the Residual Error

# Resources
* Multiple View Geometry by Hartley and Zisserman
* Computer Vision for Visual Effects by Richard Radke
* https://staff.fnwi.uva.nl/r.vandenboomgaard/IPCV20172018/LectureNotes/MATH/homogenous.html